In [1]:
import pandas as pd

In [2]:
activities = {
    'sell_date': [
        '2020-05-30', '2020-06-01', '2020-06-02',
        '2020-05-30', '2020-06-01', '2020-06-02',
        '2020-05-30'
    ],
    'product': [
        'Headphone', 'Pencil', 'Mask',
        'Basketball', 'Bible', 'Mask',
        'T-Shirt'
    ]
}



In [3]:
df = pd.DataFrame(activities)

In [20]:
df

,sell_date,product
0,2020-05-30,Headphone
1,2020-06-01,Pencil
2,2020-06-02,Mask
3,2020-05-30,Basketball
4,2020-06-01,Bible
5,2020-06-02,Mask
6,2020-05-30,T-Shirt


In [ ]:
new_df = (
    df.groupby('sell_date')['product']
      .agg(lambda x: ','.join(sorted(set(x), key=lambda s: s.lower())))
      .reset_index()
)
 
new_df['num_sold'] = new_df['product'].str.split(',').map(len)
 
new_df = new_df.rename(columns={'product': 'products'})
 
new_df = new_df[['sell_date', 'num_sold', 'products']]
new_df


,sell_date,num_sold,products
0,2020-05-30,3,"Basketball,Headphone,T-Shirt"
1,2020-06-01,2,"Bible,Pencil"
2,2020-06-02,1,Mask


In [28]:
new_df

,sell_date,num_sold,products
0,2020-05-30,3,"Basketball,Headphone,T-Shirt"
1,2020-06-01,2,"Bible,Pencil"
2,2020-06-02,1,Mask


## Logical explanation

Goal: convert the raw `df` (one row per sale) into a summary `new_df` with:
- `sell_date` — the date
- `num_sold` — number of distinct products sold on that date
- `products` — comma-separated list of distinct products (alphabetically, case‑insensitive)

How it works (high level)
1. Group rows by `sell_date`.
2. For each group, collect the `product` values, remove duplicates, sort them alphabetically ignoring case, and join them into a single string separated by commas.
3. Count the number of distinct products by splitting the joined string and taking its length.
4. Rename and reorder columns to produce the final `new_df`.

## Iterating explanation (step‑by‑step with an example)

Example input rows for `sell_date = 2020-05-30` (from your `activities` data):
- Headphone
- Basketball
- T-Shirt

Step 1 — group values
- After grouping, the group for `2020-05-30` is the sequence: `['Headphone', 'Basketball', 'T-Shirt']`.

Step 2 — remove duplicates
- Convert to a set (or use unique) to remove duplicates.
- Result: `{'Headphone', 'Basketball', 'T-Shirt'}` (unchanged here because there were no duplicates).

Step 3 — sort alphabetically, case-insensitive
- Use `sorted(..., key=lambda s: s.lower())` to ensure case-insensitive alphabetical order.
- Sorted list: `['Basketball', 'Headphone', 'T-Shirt']`.

Step 4 — join to single string
- Join with commas: `'Basketball,Headphone,T-Shirt'`.
- This becomes the `products` cell for that date.

Step 5 — count distinct products
- Split the joined string by `,`: `['Basketball', 'Headphone', 'T-Shirt']`.
- `num_sold = len(...) = 3`.

Repeat the same process for each `sell_date` group. After processing all groups, reset the index and reorder/rename columns to:
- `sell_date` | `num_sold` | `products`

## Small implementation notes and alternatives

- Current approach (in notebook):
  - groupby -> .agg(lambda x: ','.join(sorted(set(x), key=lambda s: s.lower())))
  - Then compute `num_sold` from the joined string (`str.split(',').map(len)`).
- Alternatives:
  - Use `.unique()` instead of `set(x)` to preserve first-seen order:
    - `','.join(sorted(x.unique(), key=str.lower))` (if you still want sorted, keep sorted; if you want original order, skip sorted).
  - Avoid counting from a joined string; count the unique values directly:
    - Example: compute an intermediate list column then `num_sold = new_df['products_list'].map(len)` and join only when you need the string representation.
- Edge cases to consider:
  - Missing / NaN product values — filter them out before grouping: `df = df.dropna(subset=['product'])`.
  - Products with embedded commas — if product names can contain commas, prefer storing lists or using a different delimiter or quoting.
  - Large groups — sorting cost: sorting k items costs O(k log k); grouping overall is O(n + sum k log k).

## Expected final `new_df` (based on your sample data)
sell_date | num_sold | products
---|---:|---
2020-05-30 | 3 | Basketball,Headphone,T-Shirt
2020-06-01 | 2 | Bible,Pencil
2020-06-02 | 1 | Mask

(Products shown alphabetically, case-insensitive; `num_sold` counts distinct products per date.)
